# Topic Modeling
Topic modeling is an unsupervised method that involves both dimensionality reduction and clustering text data into meaningful topics. There are several common approaches for text data. One such approach is **LSA (Latent Semantic Analysis)**. The core idea behind LSA is to take a matrix that already contains documents (tweets) and words, then decompose those into two matrices, one document (tweet) to topic matrix, and one a topic to word matrix. This allows one to find similiarity of different documents and different words. However, there are several downsides to using LSA. One is a lack of interpretability compared to the other approaches I will discuss. LSA also needs a large set of documents for accurate results. 

Another approach to discuss is **PLSA (Probabilistic latent Semantic Analysis)**. PLSA adds a probabistic spin to LSA. So each document (tweet) will have a probability of belonging in each category. However, PLSA is prone to ovefitting because it scales linearly as number of parameters increase. 

The approach used for the project will be **LDA (Latent Dirichlet Allocation)**. LDA is really just a Bayesian version of PLSA. The benefit of modeling with LDA is that it generalizes the best of the 3 approaches. This has to do with the two parameters alpha and beta which take advantage of dirichlet distributions. A deeper discussion on dirichlet distributions can be found [here](https://medium.com/@souravboss.bose/comprehensive-topic-modelling-with-nmf-lsa-plsa-lda-lda2vec-part-2-e3921e712f11). LDA will be choosen for this project mainly for its ability generalize better than the alternatives. 

In [ ]:
!pip install pyLDAvis --quiet
!pip install chart_studio --quiet


In [ ]:
import pandas as pd
import numpy as np
import time
import re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
import gensim
from spacy.tokenizer import Tokenizer
import gensim.corpora as corpora
from gensim.models.ldamulticore import LdaMulticore
from pprint import pprint
from gensim.models.coherencemodel import CoherenceModel
import plotly.express as px
import pyLDAvis.gensim
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls
from operator import itemgetter
from ipywidgets import interact
import tqdm
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets


# supress warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We will load in our preprocessed tweets from our [data cleaning notebook](https://github.com/tarrantcarter/Final_Capstone/blob/main/Data_Preprocessing_Motivational_Tweet_Generator.ipynb). The csv can be found [here](https://drive.google.com/file/d/1-d-61YuocweY0F3rSnf-idL1ixbuKq4H/view?usp=sharing). 

In [ ]:
# load in cleaned tweets from data cleaning notebook
tweets_cleaned = pd.read_json("/content/drive/MyDrive/Data/NLP_Capstone/motivational_tweets_cleaned.json")

In [ ]:
tweets_cleaned.head()

,date,user_name,content,content_preprocessed,unigram_tokens,ngrams,ngram_tokens,nouns_only,nouns_verbs,bigrams_trigrams,bigrams_trigrams_strings
44,2021-01-17 22:13:17,LewisHowes,Know this. Everything is happening for a reaso...,know happen reason favor betterment future pai...,"[know, happen, reason, favor, betterment, futu...",know_happen_reason favor betterment future pai...,"[know_happen_reason, favor, betterment, future...","[reason, favor, betterment, future, pain, feel...","[know, reason, favor, betterment, future, pain...",[know_happen_reason],know_happen_reason
61,2021-01-15 15:28:06,LewisHowes,Protect your inner peace at all costs. Create ...,protect inner peace cost create daily practice...,"[protect, inner, peace, cost, create, daily, p...",protect inner_peace cost create daily_practice...,"[protect, inner_peace, cost, create, daily_pra...","[peace, cost, practice, communicate, stress, d...","[peace, cost, practice, communicate, stress, d...","[inner_peace, daily_practice]",inner_peace daily_practice
161,2021-01-07 16:00:29,LewisHowes,Always remember to ask for exactly what you wa...,remember ask exactly want ask love good health...,"[remember, ask, exactly, want, ask, love, good...",remember ask exactly want ask love good health...,"[remember, ask, exactly, want, ask, love, good...","[health, abundance, peace, ask, wisdom, creati...","[remember, health, abundance, peace, ask, wisd...","[health_abundance, ask_wisdom]",health_abundance ask_wisdom
274,2021-01-01 02:44:46,LewisHowes,This will be your greatest year ever. All your...,great year work start pay earn happy love deep...,"[great, year, work, start, pay, earn, happy, l...",great year work start pay earn happy love deep...,"[great, year, work, start, pay, earn, happy, l...","[year, work, start, earn, love, embrace, fear,...","[year, work, start, pay, earn, love, embrace, ...",[massive_action],massive_action
317,2020-12-30 01:38:35,LewisHowes,Be grateful for the breakdown this year. It's ...,grateful breakdown year set massive breakthrou...,"[grateful, breakdown, year, set, massive, brea...",grateful breakdown year set massive breakthrou...,"[grateful, breakdown, year, set, massive, brea...","[year, breakthrough, money, mission, get, rela...","[year, set, breakthrough, money, come, mission...",[real_friend],real_friend


In [ ]:
tweets_cleaned.applymap(type)

,date,user_name,content,content_preprocessed,unigram_tokens,ngrams,ngram_tokens,nouns_only,nouns_verbs,bigrams_trigrams,bigrams_trigrams_strings
44,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
61,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
161,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
274,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
317,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...
977598,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977599,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977606,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977608,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>


In [ ]:
tweets_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 621449 entries, 44 to 977610
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   date                      621449 non-null  datetime64[ns]
 1   user_name                 621449 non-null  object        
 2   content                   621449 non-null  object        
 3   content_preprocessed      621449 non-null  object        
 4   unigram_tokens            621449 non-null  object        
 5   ngrams                    621449 non-null  object        
 6   ngram_tokens              621449 non-null  object        
 7   nouns_only                621449 non-null  object        
 8   nouns_verbs               621449 non-null  object        
 9   bigrams_trigrams          621449 non-null  object        
 10  bigrams_trigrams_strings  621449 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 56.9+ MB


In [ ]:
tweets_cleaned.shape

(621449, 11)

# Base Topic Model
In the next steps the LDA model will be created and optimized, but I haven't metioned exactly how it will be scored and optimized. Perplexity and Coherence scores will be computed initially for a base model. Our anaylsis will focus mostly on Coherence score instead of perplexity. This is mostly because Coherence score matches closer to the human intuition of topics than perplexity does. For a further discussion on perplexity vs coherence score check out the link [here](http://qpleple.com/topic-coherence-to-evaluate-topic-models/). 

In [ ]:
# create dictionary
id2word = corpora.Dictionary(tweets_cleaned['nouns_only'])
# create texts corpus
texts = tweets_cleaned['nouns_only']
# term document frequency
corpus = [id2word.doc2bow(text) for text in texts]
# print first 30 tuples from corpus
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]


In [ ]:
# number of topics
num_topics = 10
# build LDA model
base_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

In [ ]:
# filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]

# create topic sorted by 10 most relevent words
topics = [' '.join(t[0:10]) for t in words]


# print most relevent words for each topic
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
life power robert desire courage care difference question limit amp

------ Topic 1 ------
thing change einstein life albert churchill winston control end get

------ Topic 2 ------
amp dream problem feel energy fear think solve tzu obstacle

------ Topic 3 ------
mind peace moment opportunity body passion life place act define

------ Topic 4 ------
dyer life success world work person amp way failure fall

------ Topic 5 ------
look choice franklin soul henry truth life benjamin david mean

------ Topic 6 ------
day matter judge treat trust ralph life emerson hold bonaparte

------ Topic 7 ------
reality action result strength napoleon word practice abundance self power

------ Topic 8 ------
time heart john love today tomorrow bhajan anger defeat aim

------ Topic 9 ------
man way proverb create step lincoln peale experience learn jim



In [ ]:
# Compute Perplexity
## a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=tweets_cleaned['nouns_only'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -6.94619580004618

Coherence Score:  0.6164812244561871


In [ ]:
# topic distance visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(base_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.004427 -0.058685       1        1  12.182810
9      0.222194 -0.202481       2        1  11.999511
1      0.080771 -0.097124       3        1  10.576748
8      0.044011 -0.223311       4        1  10.409093
2      0.085282  0.213443       5        1   9.961151
6     -0.165460 -0.074812       6        1   9.489423
7      0.214846  0.210101       7        1   9.158813
0     -0.002775  0.180721       8        1   8.990485
3     -0.231550 -0.013117       9        1   8.732032
5     -0.242893  0.065266      10        1   8.499934, topic_info=       Term          Freq         Total Category  logprob  loglift
1336   dyer  99079.000000  99079.000000  Default  30.0000  30.0000
100    time  39704.000000  39704.000000  Default  29.0000  29.0000
60    thing  35778.000000  35778.000000  Default  28.0000  28.0000
212     man  32572.000000  32572.000000  Default  27.0000  27.0000
58    heart  19702.000000  19702.000000  Default  26.0000  26.0000
...     ...           ...           ...      ...      ...      ...
121     job   1980.628947   2721.030300  Topic10  -4.6455   2.1475
136   world   3511.207457  16506.567260  Topic10  -4.0729   0.9173
65     life   5120.974221  59430.175280  Topic10  -3.6955   0.0137
101     act   2892.138397  10758.004783  Topic10  -4.2669   1.1515
316   place   2660.878743   8474.105001  Topic10  -4.3502   1.3068

[531 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
150       1  0.000750    ability
150       2  0.000375    ability
150       3  0.002251    ability
150       4  0.000750    ability
150       5  0.598564    ability
...     ...       ...        ...
38        2  0.027483       year
38        3  0.001031       year
38        6  0.971194       year
256       4  0.998752  yesterday
3167      4  0.999774       yogi

[980 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 10, 2, 9, 3, 7, 8, 1, 4, 6])

This base model appears to be a great start. A coherence score of .61 will need to be compared to other strategies before commenting on how good it is. As can be seen in LDA visualization there is nice serperability between the 10 topics. Looking at some of the categoies some of the seem reasonable. Words like peace and mind seem like they would go togther, but it isn't clear how words like life and power go together. It should be noted that there are many first and last names appearing in the topics, but that isn't suprising since our n-grams are nouns. 

# Optimal Topic Model
In order to find the best model the focus will be on tuning 3 parameters (number of topics, alpha, and beta). Number of topics is self-explanatory. Alpha relates to document-topic density (scaled 0-1). The higher the alpha the more unique topic distribution per documents. Beta relates to topic-word density (scaled 0-1). The higher the beta the more unique word distribution per topics.

In addition to tuning these 3 parameters several different strategies will be explored as well. I will be evaluating the following approaches: n-grams only, no n-grams, noun only n-grams, noun and verb only n-grams. 

The goal it to select the strategy that has the highest cv score (metric - cv score), most seperable topics (metric - eyeballing pyLDAvis for non-overlapping topics), and most explanatory topics (metric - subjectively evaluating the topic). It is unlikely that all 3 of the goals will be met by one strategy. So exaplanatory topics will be the prefered evalutation metric, because at the end of the day I want topics that makes sense to the user. 

My hunch is that the noun and verb n-grams strategy will perfom best. Focusing on nouns makes it more likely the topic will be extracted from the text. But adding verbs makes sense because of the call-to-action nature of motivational tweets.


In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           alpha=a,
                                           eta=b,
                                           random_state=222)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
# start preprocess runtime
start_time = time.time() 

# Topics range
min_topics = 3
max_topics = 10
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = [.01,.1,.5,1,'symmetric','asymmetric']
# Beta parameter
beta = [.01,.1,.5,1,'symmetric']

# model results dict
model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=210)

    # iterate through number of topics
    for k in topics_range:
        # iterate through alpha values
        for a in alpha:
            # iterare through beta values
            for b in beta:
                # get the coherence score for the given parameters
                cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                              k=k, a=a, b=b)
                # Save the model results
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)
                
                pbar.update(1)
    lda_tuning_results = pd.DataFrame(model_results)                
    lda_tuning_results.to_csv('/content/drive/MyDrive/Data/NLP_Capstone/lda_tuning_nouns_3-10_topics.csv', index=False)
    pbar.close()

# print preprocess runtime
print(time.strftime(f'%H hours, %M minutes, %S seconds', time.gmtime(time.time() - start_time)))

100%|██████████| 210/210 [3:59:55<00:00, 68.55s/it]

03 hours, 59 minutes, 55 seconds


In [ ]:
lda_tuning_results.sort_values('Coherence',ascending=False).head(30)

,Topics,Alpha,Beta,Coherence
160,8,0.5,0.01,0.623141
205,9,asymmetric,0.01,0.616538
161,8,0.5,0.1,0.615413
185,9,0.1,0.01,0.614666
206,9,asymmetric,0.1,0.614617
191,9,0.5,0.1,0.613899
176,8,asymmetric,0.1,0.612633
190,9,0.5,0.01,0.611294
179,8,asymmetric,symmetric,0.611154
164,8,0.5,symmetric,0.610316


It seems that 8-9 topics are the sweet spot for these particular parameters. I will withold judgement before speculating on what these particular resuts means until all the other strategies can be analyzed. These results will be compared to other strategies in the [optimized model deployment notebook](https://github.com/tarrantcarter/Final_Capstone/blob/main/Optimal_Deployed_Motivational_Tweet_Generator.ipynb). 

In [ ]:
# number optimal of topics
num_topics = 8
# build optimal LDA model
optimal_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                        alpha=.5,
                                        eta=.01,
                                        random_state=222
                                        )
# filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in optimal_model.print_topics()]

# create topic sorted by 10 most relevent words
topics = [' '.join(t[0:10]) for t in words]


# print most relevent words for each topic
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
heart mind place matter john step opportunity learn franklin lincoln

------ Topic 1 ------
strength body problem einstein reality energy result peale treat respect

------ Topic 2 ------
dyer time day jim wisdom rohn decision amp experience william

------ Topic 3 ------
act dream action fall courage question word mean habit god

------ Topic 4 ------
person power choice moment judge fear truth try care attract

------ Topic 5 ------
life way look create today george presence goal tomorrow water

------ Topic 6 ------
thing amp love peace work happiness feel passion trust ability

------ Topic 7 ------
man change world success proverb failure churchill winston robert henry



In [ ]:
# Compute Perplexity
## a measure of how good the model is. lower the better
base_perplexity = optimal_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=optimal_model, texts=tweets_cleaned['nouns_only'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -27.045391793374172

Coherence Score:  0.5995881610113447


In [ ]:
# topic distance visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.133034  0.001637       1        1  13.510364
7      0.154800  0.187030       2        1  13.317380
5     -0.010597  0.203740       3        1  13.080425
0     -0.328622 -0.061305       4        1  12.614480
6     -0.075244 -0.230418       5        1  12.573215
2      0.241817 -0.141424       6        1  12.010494
4      0.156400 -0.183022       7        1  11.641429
3     -0.005522  0.223761       8        1  11.252213, topic_info=             Term           Freq          Total Category  logprob  loglift
1336         dyer  130122.000000  130122.000000  Default  30.0000  30.0000
65           life   77764.000000   77764.000000  Default  29.0000  29.0000
60          thing   46190.000000   46190.000000  Default  28.0000  28.0000
212           man   41331.000000   41331.000000  Default  27.0000  27.0000
100          time   32426.000000   32426.000000  Default  26.0000  26.0000
...           ...            ...            ...      ...      ...      ...
102          kind    5338.472737    5593.166200   Topic8  -3.9345   2.1380
302   expectation    3896.931033    4036.784081   Topic8  -4.2492   2.1493
215          want    4752.529694    5021.389490   Topic8  -4.0507   2.1296
885          book    2858.697948    3038.949041   Topic8  -4.5590   2.1235
990       emotion    2885.362092    3124.868652   Topic8  -4.5497   2.1049

[510 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
150       1  0.001713  ability
150       2  0.000311  ability
150       3  0.000311  ability
150       5  0.988305  ability
150       6  0.000156  ability
...     ...       ...      ...
38        2  0.001831     year
38        3  0.000366     year
38        5  0.015376     year
38        6  0.929178     year
38        7  0.003661     year

[1365 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 8, 6, 1, 7, 3, 5, 4])